In [ ]:
# Import libraries and query Aletheia
# May take 1-4 minutes to run depending on query parameters

import time
import numpy as np
import pandas as pd
from datetime import datetime as dt
from brtdevkit.core.db import DBConnector, DatetimeFilter
from matplotlib import pyplot as plt, rcParams
import seaborn as sns
sns.set_style('whitegrid')

def get_shasta_data(filters={}, start=None, end=None, limit=None):
    """
    Query relevant Shasta data for calculations. Written by Andrei P.
    """
    start_time = time.time()
    connector = DBConnector()
    img_filters = {'project_name': 'shasta', **filters}
    if start is not None or end is not None:
        img_filters = [img_filters, DatetimeFilter(key="collected_on", start=start, end=end)]
    df = connector.get_documents_df('image', img_filters, limit=limit)
    elapsed_time = time.time() - start_time
    return df, elapsed_time

# Set filters for query
filters = {"artifacts.kind": "nrg", 
           "crop_name": {"$in": ['CORN', 'COTTON', 'SOYBEANS']}, 
           "robot_name": {"$in": ["DCM-MANATEE","DCM-OTTER", "DCM-DOLPHIN", "DCM-WALRUS", "DCM-PORPOISE", "DCM-SEAL"]} }

# Set start date
start = dt(2020, 3,1)

full_df, elapsed_time = get_shasta_data(filters=filters, start=start)
full_df['date_collected'] = pd.to_datetime(full_df['collected_on'].dt.date)
print(f"Queried {len(full_df)} images in {elapsed_time:.2f} s.")

In [ ]:
print(full_df.tillage_practice.unique())

field_names = ['Cook Farm', 'Dave north south', 'Franzen', '6.5 Acres', 'Kixmiller']
field_recalls = [0.71316,0.34483,  0.8126, 0.3276, 0.71321]


plt.figure(figsize=(10,6))
plt.title('Small Crop, High Residue Fields')
plt.bar(field_names, field_recalls, color ='plum')
plt.ylabel('Grid Instance Recall')
#plt.savefig('high_residue.png')
plt.show()